In [247]:
import sys, os
sys.path.append("../../")
import numpy as np
import lightgbm as lgb
import papermill as pm
import pandas as pd
import category_encoders as ce
from tempfile import TemporaryDirectory
from sklearn.metrics import roc_auc_score, log_loss
#import scrapbook as sb

import reco_utils.recommender.lightgbm.lightgbm_utils as lgb_utils
import reco_utils.dataset.criteo as criteo

print("System version: {}".format(sys.version))
print("LightGBM version: {}".format(lgb.__version__))

System version: 3.6.10 |Anaconda, Inc.| (default, May  7 2020, 23:06:31) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
LightGBM version: 2.2.1


In [207]:
# data = 'train.gz'
# data_full = pd.read_csv(data, sep=',', compression='gzip')

In [208]:
# import featuretools as ft
# import os
# from tqdm import tqdm


# def make_user_sample(all_data, user_ids, out_dir):
#     all_data_sample = all_data[all_data["id"].isin(user_ids)]

#     try:
#         os.mkdir(out_dir)
#     except:
#         pass
#     all_data_sample.to_csv(os.path.join(out_dir, "all_data.csv"), index=None)

# def main():
#     all_data = pd.read_csv(data, sep=',', compression='gzip')
#     users_unique = all_data["id"].unique()
#     chunksize = 500000
#     part_num = 0
#     partition_dir = "data"
#     try:
#         os.mkdir(partition_dir)
#     except:
#         pass
#     for i in tqdm(range(0, len(users_unique), chunksize)):
#         users_keep = users_unique[i: i+chunksize]
#         make_user_sample(all_data, users_keep,os.path.join(partition_dir, "part_%d" % part_num))
#         part_num += 1

# if __name__ == "__main__":
#     main()

In [209]:
all_data = pd.read_csv('data/part_0/all_data.csv')

In [210]:
pd.set_option('display.float_format', '{:.0f}'.format)
all_data.head(5)

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1000009418151094400,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,10000169349117863936,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,10000371904215119872,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,10000640724480837632,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,10000679056417042432,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [211]:
all_data.site_category.unique()

array(['28905ebd', '0569f928', 'f028772b', '50e219e0', '3e814130',
       '76b2941d', 'f66779e6', '335d28a8', '72722551', '75fa27f6',
       '110ab22d', 'c0dd3be3', 'bcf865d9', 'a818d37a', '42a36e14',
       'e787de0e', '5378d028', '9ccfa2ea', '70fb0e29', 'dedf689d',
       '8fd0aea4'], dtype=object)

In [212]:
temp1 = all_data

In [213]:
temp1['id'] = np.arange(len(temp1))

In [214]:
def conditions(s):
    if (s['site_category'] == '28905ebd') :
        return 'Egg & Cheese Burger'
    elif (s['site_category'] == '0569f928'):
        return 'Veggie Egg White Burger'
    elif (s['site_category'] == 'f028772b'):
        return 'Sausage egg & cheese burger'
    elif (s['site_category'] == '50e219e0'):
        return 'Bacon egg & cheese burger'
    elif (s['site_category'] == '3e814130'):
        return 'Turkey Sausage Burger'
    elif (s['site_category'] == '76b2941d'):
        return 'Ham egg & cheese burger'
    elif (s['site_category'] == 'f66779e6'):
        return 'Egg & cheese wrap'
    elif (s['site_category'] == '335d28a8'):
        return 'Hot Coffee'
    elif (s['site_category'] == '72722551'):
        return 'Cold Coffee'
    elif (s['site_category'] == '75fa27f6'):
        return 'Hot Chocolate'
    elif (s['site_category'] == '110ab22d'):
        return 'Cappuccino'
    elif (s['site_category'] == 'c0dd3be3'):
        return 'Hot Tea'
    elif (s['site_category'] == 'bcf865d9'):
        return 'Iced Tea'
    elif (s['site_category'] == 'a818d37a'):
        return 'Americano'
    elif (s['site_category'] == '42a36e14'):
        return 'Latte'
    elif (s['site_category'] == 'e787de0e'):
        return 'Macchiato'
    elif (s['site_category'] == '5378d028'):
        return 'Frozen Chocolate'
    else:
        return 'Others'

In [215]:
def sites(s):
    if (s['site_category'] == '28905ebd') :
        return 'Snackfair'
    elif (s['site_category'] == '0569f928'):
        return 'Instacart'
    elif (s['site_category'] == 'f028772b'):
        return 'Amazon'
    elif (s['site_category'] == '50e219e0'):
        return 'Stop&Shop'
    elif (s['site_category'] == '3e814130'):
        return 'Target'
    elif (s['site_category'] == '76b2941d'):
        return 'Snackfair'
    elif (s['site_category'] == 'f66779e6'):
        return 'Instacart'
    elif (s['site_category'] == '335d28a8'):
        return 'Amazon'
    elif (s['site_category'] == '72722551'):
        return 'Stop&Shop'
    elif (s['site_category'] == '75fa27f6'):
        return 'Target'
    elif (s['site_category'] == '110ab22d'):
        return 'Snackfair'
    elif (s['site_category'] == 'c0dd3be3'):
        return 'Target'
    elif (s['site_category'] == 'bcf865d9'):
        return 'Amazon'
    elif (s['site_category'] == 'a818d37a'):
        return 'Stop&Shop'
    elif (s['site_category'] == '42a36e14'):
        return 'Instacart'
    elif (s['site_category'] == 'e787de0e'):
        return 'Amazon'
    elif (s['site_category'] == '5378d028'):
        return 'Snackfair'
    else:
        return 'Others'

In [216]:
temp1['product_cat'] = temp1.apply(conditions, axis=1)

In [217]:
temp1['product_cat']

0                 Egg & Cheese Burger
1                 Egg & Cheese Burger
2                 Egg & Cheese Burger
3                 Egg & Cheese Burger
4             Veggie Egg White Burger
                     ...             
499995      Bacon egg & cheese burger
499996    Sausage egg & cheese burger
499997      Bacon egg & cheese burger
499998    Sausage egg & cheese burger
499999    Sausage egg & cheese burger
Name: product_cat, Length: 500000, dtype: object

In [218]:
temp1['product_cat'] = temp1.apply(conditions, axis=1)

In [219]:
temp1['Site Name'] = temp1.apply(sites, axis=1)

In [220]:
def actions(s):
    if (s['Site Name'] == 'Snackfair') :
        return 1
    else:
        return 0

In [221]:
temp1['Action'] =temp1.apply(actions, axis=1)

In [222]:
temp1['Action'].value_counts()

0    348720
1    151280
Name: Action, dtype: int64

In [223]:
all_data = temp1

In [224]:
MAX_LEAF = 64
MIN_DATA = 20
NUM_OF_TREES = 100
TREE_LEARNING_RATE = 0.15
EARLY_STOPPING_ROUNDS = 20
METRIC = "auc"
SIZE = "sample"

In [225]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'num_class': 1,
    'objective': "binary",
    'metric': METRIC,
    'num_leaves': MAX_LEAF,
    'min_data': MIN_DATA,
    'boost_from_average': True,
    #set it according to your cpu cores.
    'num_threads': 20,
    'feature_fraction': 0.8,
    'learning_rate': TREE_LEARNING_RATE,
}

In [226]:
# split data to 3 sets    
length = len(all_data)
train_data = all_data.loc[:0.8*length-1]
valid_data = all_data.loc[0.8*length:0.9*length-1]
test_data = all_data.loc[0.9*length:]

In [227]:
test_data.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C15,C16,C17,C18,C19,C20,C21,product_cat,Site Name,Action
450000,450000,0,14102102,1005,1,d9750ee7,98572c79,f028772b,ecad2386,7801e8d9,...,320,50,1993,2,1063,-1,33,Sausage egg & cheese burger,Ask.com,0
450001,450001,0,14102102,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,320,50,1722,0,35,-1,79,Egg & Cheese Burger,Snackfair,1
450002,450002,0,14102102,1005,0,85f751fd,c4e18dd6,50e219e0,febd1138,82e27996,...,320,50,2480,3,297,100111,61,Bacon egg & cheese burger,Yahoo,0
450003,450003,0,14102102,1005,0,85f751fd,c4e18dd6,50e219e0,c51f82bc,d9b5648e,...,320,50,2480,3,297,100111,61,Bacon egg & cheese burger,Yahoo,0
450004,450004,1,14102102,1005,0,5b08c53b,7687a86e,3e814130,ecad2386,7801e8d9,...,300,250,1994,2,39,-1,33,Turkey Sausage Burger,Bing,0


In [228]:
nume_cols = ['Action','id','hour','banner_pos','device_type','device_conn_type','C14','C15', 'C16','C17','C18','C19','C20','C21']
cate_cols=['Site Name','site_id','site_domain','site_category','product_cat','app_id','app_domain','app_category','device_id','device_ip','device_model']
label_col='click'
ord_encoder = ce.ordinal.OrdinalEncoder(cols=cate_cols)

def encode_csv(df, encoder, label_col, typ='fit'):
    if typ == 'fit':
        df = encoder.fit_transform(df)
    else:
        df = encoder.transform(df)
    y = df[label_col].values
    del df[label_col]
    return df, y

train_x, train_y = encode_csv(train_data, ord_encoder, label_col)
valid_x, valid_y = encode_csv(valid_data, ord_encoder, label_col, 'transform')
test_x, test_y = encode_csv(test_data, ord_encoder, label_col, 'transform')

print('Train Data Shape: X: {trn_x_shape} ; Y: {trn_y_shape}.\nValid Data Shape: X: {vld_x_shape}; Y: {vld_y_shape}.\nTest Data Shape: X: {tst_x_shape}; Y: {tst_y_shape}.\n'
      .format(trn_x_shape=train_x.shape,
              trn_y_shape=train_y.shape,
              vld_x_shape=valid_x.shape,
              vld_y_shape=valid_y.shape,
              tst_x_shape=test_x.shape,
              tst_y_shape=test_y.shape,))
train_x.head()

Train Data Shape: X: (400000, 26) ; Y: (400000,).
Valid Data Shape: X: (50000, 26); Y: (50000,).
Test Data Shape: X: (50000, 26); Y: (50000,).



,id,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,C15,C16,C17,C18,C19,C20,C21,product_cat,Site Name,Action
0,0,14102100,1005,0,1,1,1,1,1,1,...,320,50,1722,0,35,-1,79,1,1,1
1,1,14102100,1005,0,1,1,1,1,1,1,...,320,50,1722,0,35,100084,79,1,1,1
2,2,14102100,1005,0,1,1,1,1,1,1,...,320,50,1722,0,35,100084,79,1,1,1
3,3,14102100,1005,0,1,1,1,1,1,1,...,320,50,1722,0,35,100084,79,1,1,1
4,4,14102100,1005,1,2,2,2,1,1,1,...,320,50,2161,0,35,-1,157,2,2,0


In [233]:
train_x.to_csv('train_x.csv')

In [234]:
lgb_train = lgb.Dataset(train_x, train_y.reshape(-1), params=params, categorical_feature=cate_cols)
lgb_valid = lgb.Dataset(valid_x, valid_y.reshape(-1), reference=lgb_train, categorical_feature=cate_cols)
lgb_test = lgb.Dataset(test_x, test_y.reshape(-1), reference=lgb_train, categorical_feature=cate_cols)
lgb_model = lgb.train(params,
                      lgb_train,
                      num_boost_round=NUM_OF_TREES,
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                      valid_sets=lgb_valid,
                      categorical_feature=cate_cols)

[1]	valid_0's auc: 0.743346
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's auc: 0.759747
[3]	valid_0's auc: 0.763082
[4]	valid_0's auc: 0.764018
[5]	valid_0's auc: 0.767116
[6]	valid_0's auc: 0.768268
[7]	valid_0's auc: 0.769127
[8]	valid_0's auc: 0.769834
[9]	valid_0's auc: 0.770249
[10]	valid_0's auc: 0.770997
[11]	valid_0's auc: 0.771853
[12]	valid_0's auc: 0.772364
[13]	valid_0's auc: 0.772821
[14]	valid_0's auc: 0.773813
[15]	valid_0's auc: 0.774261
[16]	valid_0's auc: 0.774636
[17]	valid_0's auc: 0.774975
[18]	valid_0's auc: 0.775613
[19]	valid_0's auc: 0.775769
[20]	valid_0's auc: 0.775809
[21]	valid_0's auc: 0.776072
[22]	valid_0's auc: 0.776205
[23]	valid_0's auc: 0.776482
[24]	valid_0's auc: 0.776716
[25]	valid_0's auc: 0.77676
[26]	valid_0's auc: 0.776873
[27]	valid_0's auc: 0.776933
[28]	valid_0's auc: 0.777036
[29]	valid_0's auc: 0.777004
[30]	valid_0's auc: 0.777256
[31]	valid_0's auc: 0.777236
[32]	valid_0's auc: 0.777369
[33]	valid_0's auc: 

In [235]:
test_preds = lgb_model.predict(test_x)
auc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
logloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds), eps=1e-12)
res_basic = {"auc": auc, "logloss": logloss}
print(res_basic)
pm.record("res_basic", res_basic)

{'auc': 0.7460370838761532, 'logloss': 0.39134711769783576}


/Users/sumedhtelang/opt/anaconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  


In [236]:
label_col = 'click'
num_encoder = lgb_utils.NumEncoder(cate_cols, nume_cols, label_col)
train_x, train_y = num_encoder.fit_transform(train_data)
valid_x, valid_y = num_encoder.transform(valid_data)
test_x, test_y = num_encoder.transform(test_data)
del num_encoder
print('Train Data Shape: X: {trn_x_shape}; Y: {trn_y_shape}.\nValid Data Shape: X: {vld_x_shape}; Y: {vld_y_shape}.\nTest Data Shape: X: {tst_x_shape}; Y: {tst_y_shape}.\n'
      .format(trn_x_shape=train_x.shape,
              trn_y_shape=train_y.shape,
              vld_x_shape=valid_x.shape,
              vld_y_shape=valid_y.shape,
              tst_x_shape=test_x.shape,
              tst_y_shape=test_y.shape,))

2020-07-31 17:04:34,234 [INFO] Filtering and fillna features
100%|██████████| 14/14 [00:00<00:00, 185.04it/s]
2020-07-31 17:04:37,863 [INFO] Ordinal encoding cate features
2020-07-31 17:04:39,015 [INFO] Target encoding cate features
100%|██████████| 11/11 [00:08<00:00,  1.24it/s]
2020-07-31 17:04:47,879 [INFO] Start manual binary encoding
100%|██████████| 11/11 [00:04<00:00,  2.62it/s]
2020-07-31 17:05:01,168 [INFO] Filtering and fillna features
100%|██████████| 14/14 [00:00<00:00, 1988.90it/s]
2020-07-31 17:05:01,363 [INFO] Ordinal encoding cate features
2020-07-31 17:05:01,428 [INFO] Target encoding cate features
100%|██████████| 11/11 [00:00<00:00, 11.30it/s]
2020-07-31 17:05:02,404 [INFO] Start manual binary encoding
100%|██████████| 11/11 [00:02<00:00,  5.49it/s]
2020-07-31 17:05:10,533 [INFO] Filtering and fillna features
100%|██████████| 14/14 [00:00<00:00, 1908.11it/s]
2020-07-31 17:05:10,719 [INFO] Ordinal encoding cate features
2020-07-31 17:05:10,784 [INFO] Target encoding c

Train Data Shape: X: (400000, 119); Y: (400000, 1).
Valid Data Shape: X: (50000, 119); Y: (50000, 1).
Test Data Shape: X: (50000, 119); Y: (50000, 1).



In [237]:
lgb_train = lgb.Dataset(train_x, train_y.reshape(-1), params=params)
lgb_valid = lgb.Dataset(valid_x, valid_y.reshape(-1), reference=lgb_train)
lgb_model = lgb.train(params,
                      lgb_train,
                      num_boost_round=NUM_OF_TREES,
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                      valid_sets=lgb_valid)

[1]	valid_0's auc: 0.760085
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's auc: 0.766221
[3]	valid_0's auc: 0.765887
[4]	valid_0's auc: 0.768127
[5]	valid_0's auc: 0.769638
[6]	valid_0's auc: 0.769821
[7]	valid_0's auc: 0.770811
[8]	valid_0's auc: 0.771186
[9]	valid_0's auc: 0.771214
[10]	valid_0's auc: 0.772052
[11]	valid_0's auc: 0.772359
[12]	valid_0's auc: 0.773194
[13]	valid_0's auc: 0.77334
[14]	valid_0's auc: 0.773847
[15]	valid_0's auc: 0.774184
[16]	valid_0's auc: 0.774139
[17]	valid_0's auc: 0.774129
[18]	valid_0's auc: 0.774144
[19]	valid_0's auc: 0.774157
[20]	valid_0's auc: 0.774233
[21]	valid_0's auc: 0.77107
[22]	valid_0's auc: 0.771349
[23]	valid_0's auc: 0.771593
[24]	valid_0's auc: 0.771754
[25]	valid_0's auc: 0.771712
[26]	valid_0's auc: 0.772099
[27]	valid_0's auc: 0.772107
[28]	valid_0's auc: 0.771981
[29]	valid_0's auc: 0.77169
[30]	valid_0's auc: 0.77177
[31]	valid_0's auc: 0.771817
[32]	valid_0's auc: 0.771739
[33]	valid_0's auc: 0.7

In [238]:
test_preds = lgb_model.predict(test_x)
auc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
logloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds), eps=1e-12)
res_optim = {"auc": auc, "logloss": logloss}
print(res_optim)
pm.record("res_optim", res_optim)

{'auc': 0.7481931784465777, 'logloss': 0.3901306634357624}


/Users/sumedhtelang/opt/anaconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  


In [239]:
test_preds

array([0.51116256, 0.21900024, 0.06010167, ..., 0.20061591, 0.10809316,
       0.20365727])

In [240]:
#with TemporaryDirectory() as tmp:
save_file = (r'finished.model')
lgb_model.save_model(save_file)
loaded_model = lgb.Booster(model_file=save_file)

# eval the performance again
test_preds = loaded_model.predict(test_x)

auc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
logloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds), eps=1e-12)
print({"auc": auc, "logloss": logloss})

{'auc': 0.7481931784465777, 'logloss': 0.3901306634357624}
